In [61]:
import numpy as np
import scipy.io as sc
import matplotlib.pyplot as plt

from shapely.geometry import Polygon, LineString, Point

from IPython.display import clear_output
import time 

from particle_resampling import *

In [62]:
plt.rcParams["figure.figsize"] = (30, 10)

In [63]:
def plotCarte(m):
    plt.plot(m[0,:],m[1,:],'b-')
    plt.xlabel('Coordonnées en x (m)')
    plt.ylabel('Coordonnées en y (m)')

def gauss(x, s):
    return np.exp(-(x**2)/(2*s**2))/np.sqrt(2*np.pi*s**2)

def euclidian_distance(x1,y1,x2,y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

def wrap2pi(theta):
    return np.arctan2(np.sin(theta), np.cos(theta))

In [64]:
NumeroDeQuestion=1

In [65]:
# Carte du monde, connue du robot, modélisée par des segments de droite.
matlab_data = sc.loadmat('Carte.mat')
carte = matlab_data["Carte"]

In [66]:
pCarte = Polygon([carte[:, i] for i in range(np.size(carte, 1))]) 

In [67]:
if (NumeroDeQuestion == 1):
    matlab_data = sc.loadmat('Q1Trajectoire.mat')
elif (NumeroDeQuestion == 2):
    matlab_data = sc.loadmat('Q2Trajectoire.mat')

In [68]:
# On récupère les variables de matlab
xPose = np.reshape(matlab_data["xPose"],-1)
yPose = np.reshape(matlab_data["yPose"],-1)
omega = np.reshape(matlab_data["omega"],-1)
dT    = matlab_data["dT"]
V     = np.reshape(matlab_data["V"],-1)
Lidar = matlab_data["Lidar"]
anglePose = np.reshape(matlab_data["anglePose"],-1)
Compas    = np.reshape(matlab_data["Compas"],-1)

AngleLiDAR = np.array([0.0, 0.5*np.pi, np.pi, 1.5*np.pi])
nStep = np.size(V)        # Nombre de pas dans la simulation

In [69]:
# Paramètres du filtre. À vous de les ajuster !
nParticules = 1000
Somega  = 0.04
SV      = 0.1
Scompas = 0.6
SLiDAR = 0.5

RatioResampling = 0.4

In [73]:
AX0 = np.zeros(nStep)
AX1 = np.zeros(nStep)
AX2 = np.zeros(nStep)
ATime = np.zeros(nStep)


In [74]:
LIDAR_RANGE = 20
def mesure(X):
    res = []

    for iSensor in range(0, np.size(Lidar, 0)):
        AngleLiDARGlobal = wrap2pi(X[2] + AngleLiDAR[iSensor])
        l = LineString([(X[0], X[1]), (X[0]+LIDAR_RANGE*np.cos(AngleLiDARGlobal), X[1]+LIDAR_RANGE*np.sin(AngleLiDARGlobal))])

        inter = l.intersection(pCarte.buffer(0))
        if not inter.is_empty:
            if inter.geom_type == 'MultiLineString':
                dist = []
                for line in inter.geoms:
                    dist += [euclidian_distance(X[0], X[1], p[0], p[1]) 
                             for p in line.coords 
                             if Point(X[0], X[1]) != Point(p[0], p[1])]
            else:
                dist = [euclidian_distance(X[0], X[1], p[0], p[1]) 
                        for p in inter.coords 
                        if Point(X[0], X[1]) != Point(p[0], p[1])]
            zhat = min(dist)
        else:
            zhat = LIDAR_RANGE
        res.append(zhat)
    return res
    

In [ ]:
nMeans = 10
AMeans = np.zeros(nMeans)

for iMean in range(nMeans):
    SLiDAR = (1+iMean)*0.1

    # Initialiser les particules au hasard
    X = np.zeros((3,nParticules))
    X[0,:] = 19*np.random.rand(nParticules) # position en x
    X[1,:] = 7*np.random.rand(nParticules)  # position en y
    X[2,:] = 2.0*np.pi*np.random.rand(nParticules) # orientation
    w = np.ones(nParticules) # Poids égaux pour les particules

    plotCarte(carte)

    plt.scatter(X[0,:],X[1,:],marker='o',color='k',s=7)
    for iParticule in range(0, np.size(X,1)):
        for iSensor in range(0, np.size(AngleLiDAR)):
            AngleLiDARGlobal = wrap2pi(X[2, iParticule] + AngleLiDAR[iSensor])
            # plt.plot(X[0, iParticule]+[0, np.cos(AngleLiDARGlobal)*Lidar[iSensor,0]],
            #          X[1, iParticule]+[0, np.sin(AngleLiDARGlobal)*Lidar[iSensor,0]],'r')

    plt.axis('equal')
    plt.show()

    # ============== Boucle du filtre a particules =================
    for iStep in range(0, nStep):

        # ================ Début du filtre a particules =============
        for iParticule in range(0,np.size(X,1)):
            # Prédiction
            X[2, iParticule] = wrap2pi(Compas[iStep] + Scompas*np.random.randn())
            Vused = V[iStep] + SV*np.random.randn()
            X[0, iParticule] = X[0, iParticule] + Vused*np.cos(X[2, iParticule])*dT
            X[1, iParticule] = X[1, iParticule] + Vused*np.sin(X[2, iParticule])*dT

            zhat = mesure(X[:, iParticule])
            
            # Mise-à-jour
            for iSensor in range(0, np.size(Lidar, 0)):
                wnew = gauss(zhat[iSensor]-Lidar[iSensor,iStep], SLiDAR)
                w[iParticule] = w[iParticule]*wnew

            X, w = particle_resampling(X, w, RatioResampling)   # Ré-échantillonnage

        # ================ Fin du filtre a particules =============

        AX0[iStep]      = sum(X[0,:]*w)
        AX1[iStep]      = sum(X[1,:]*w)
        AX2[iStep]      = sum(X[2,:]*w)
        ATime[iStep]    = iStep*dT


        clear_output(wait=True)

        print("iStep", iStep)
        
        plotCarte(carte)
        
        plt.scatter(X[0,:],X[1,:],marker='o',color='k',s=7, label="Position estimée")

        for iSensor in range(0, np.size(AngleLiDAR)):
            AngleLiDARReel = wrap2pi(anglePose[iStep] + AngleLiDAR[iSensor])
            plt.plot(xPose[iStep]+[0, np.cos(AngleLiDARReel)*Lidar[iSensor, iStep]],
                    yPose[iStep]+[0, np.sin(AngleLiDARReel)*Lidar[iSensor, iStep]], 'g')
        
            AngleLiDARGlobal = wrap2pi(AX2[iStep] + AngleLiDAR[iSensor])
            plt.plot(AX0[iStep]+[0, np.cos(AngleLiDARGlobal)*Lidar[iSensor, iStep]],
                    AX1[iStep]+[0, np.sin(AngleLiDARGlobal)*Lidar[iSensor, iStep]],'r')

        plt.xlabel("Temps(s)")
        plt.ylabel("Estimé de la position (m)")

        plt.legend()

        plt.show()
    
    AMeans[iMean] = np.mean(np.abs(xPose[:nStep]-AX0))

plt.plot(0.1*np.array(range(1, nMeans+1)), AMeans)

plt.xlabel("SLiDAR(m)")
plt.ylabel("Erreur moyenne")

plt.show()
